<a href="https://colab.research.google.com/github/yujin-jo/mini-project-team1/blob/main/analysis_04_%EB%B0%B0%EA%B8%89%EC%82%AC%EB%B9%84%EC%A4%91%EB%B3%80%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기본 세팅
- 깃허브 클론 
- 드라이브 마운트 
- 파일 임포트

In [ ]:
!git clone https://github.com/yujin-jo/mini-project-team1.git

Cloning into 'mini-project-team1'...
remote: Enumerating objects: 588, done.
remote: Counting objects: 100% (588/588), done.
remote: Compressing objects: 100% (552/552), done.
remote: Total 588 (delta 167), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (588/588), 807.79 KiB | 5.65 MiB/s, done.
Resolving deltas: 100% (167/167), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd 
import numpy as   np
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols

In [ ]:
movies = pd.read_csv("/content/mini-project-team1/1. Crawling/data/KOBIS_TOT (1).csv")
movies["개봉일"] = pd.to_datetime(movies["개봉일"])
movies.sample(10)

,순번,개봉연도,영화명,감독,배급사,개봉일,국적,전국스크린수,전국매출액,전국관객수,장르,등급,영화구분
502,503,2013,슈퍼배드 2,"크리스 리노드,피에르 꼬팽",유니버설픽쳐스인터내셔널 코리아(유),2013-09-12,미국,421,6359208000,925108,애니메이션,전체관람가,일반영화
1721,1722,2016,부릉! 부릉! 브루미즈: 스피더의 모험 일기,이영준,(주)삼지애니메이션,2016-12-15,한국,194,81128500,10208,애니메이션,전체관람가,독립/예술영화
1313,1314,2015,어떤살인,안용훈,"TCO(주)더콘텐츠온,(주)전망좋은 영화사",2015-10-28,한국,179,137025800,17190,스릴러,청소년관람불가,독립/예술영화
2481,2482,2018,킨: 더 비기닝,"조나단 베이커,조쉬 베이커",판씨네마(주),2018-10-11,미국,167,151798300,17543,미스터리,15세이상관람가,일반영화
1253,1254,2015,미안해 사랑해 고마워,전윤수,(주)쇼박스,2015-10-28,한국,344,359247800,51207,드라마,15세이상관람가,일반영화
1826,1827,2017,분노의 질주: 더 익스트림,F. 게리 그레이,유니버설픽쳐스인터내셔널 코리아(유),2017-04-12,미국,1469,29981156348,3653238,액션,15세이상관람가,일반영화
3144,3145,2020,비밀정보원: 인 더 프리즌,안드레아 디 스테파노,(주)제이앤씨미디어그룹,2020-04-15,영국,200,172588940,20317,범죄,청소년관람불가,일반영화
2863,2864,2019,경계선,알리 아바시,(주)제이앤씨미디어그룹,2019-10-24,스웨덴,48,38676620,17462,판타지,청소년관람불가,독립/예술영화
2280,2281,2018,궁합,홍창표,(주)씨제이이엔엠,2018-02-28,한국,966,10478875438,1340117,코미디,12세이상관람가,일반영화
2815,2816,2019,주전장,미키 데자키,(주)시네마달,2019-07-25,미국,56,35376040,39980,다큐멘터리,전체관람가,독립/예술영화


---
# I. 가설 설정과 분석 배경

1. 가설  :  2011년 ~ 2020년까지 국내 영화 배급사에서 CJ E&M의 비중은 연도 별로 감소했는가? 
2. 가설을 선택한 이유 : 
    - 기존에는 국내 영화의 경우 CJ E&M이 독점했왔음. 
    - 다른 신생 배급사들은 없었을까 궁금했음 
    -  기존에 있던 영세 배급사들은 성장하거나 망했을까 궁금했고, 여전히 CJ E&M이 영화 산업을 지배하고 있는지 확인하고 싶었음 

# II. 분석 방향 
1. 활용 데이터 
  - 영화 개봉연도 : 연도 별로 영화 수 집계 필요 
  - 영화 배급사 : 영화 별 배급사 정보 (KOBIS 참고) 
  - 영화 수 : 연도 별, 배급사 별로 집계한 영화 수

2. 분석 방법론 
  - 시계열 분석 활용 (2011 ~ 2020년 영화 정보) 
  - 영화 배급사의 종류가 연도 별로 어떻게 달라졌는지 확인  (중소 배급사 등 배급사가 늘었는지 확인하기 위함)
  - 개봉 연도와 제작사 별로 영화 편수를 카운트한다. 
  - 연도 별로 제작사 별 비중을 계산하여 추이를 확인한다. 

3. 시각화 
  - 비율 바 차트




# III. 분석 과정  

## 1. 데이터 테이블 분석
- 2011년 ~ 2020년까지의 총 3,375개의 영화 데이터가 들어감 
- movie 테이블에 들어간 정보를 movies.info()로 파악 
- 영화 배급사의 종류는 434개로 파악되나, 두 개 이상의 영화사가 있음 

In [ ]:
# 영화 기본 정보 확인 
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3375 entries, 0 to 3374
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   순번      3375 non-null   int64         
 1   개봉연도    3375 non-null   int64         
 2   영화명     3375 non-null   object        
 3   감독      3375 non-null   object        
 4   배급사     3374 non-null   object        
 5   개봉일     3375 non-null   datetime64[ns]
 6   국적      3375 non-null   object        
 7   전국스크린수  3375 non-null   int64         
 8   전국매출액   3375 non-null   int64         
 9   전국관객수   3375 non-null   int64         
 10  장르      3375 non-null   object        
 11  등급      3375 non-null   object        
 12  영화구분    3375 non-null   object        
dtypes: datetime64[ns](1), int64(5), object(7)
memory usage: 342.9+ KB


In [ ]:
#value_count를 했을 때 하나의 영화에 두 개 이상의 배급사가 있는 것을 확인 -> 새로운 전처리가 필요하다. 
movies['배급사'].value_counts()

(주)씨제이이엔엠                             215
롯데쇼핑㈜롯데엔터테인먼트                         165
유니버설픽쳐스인터내셔널 코리아(유)                   148
(주)넥스트엔터테인먼트월드(NEW)                   138
(주)팝엔터테인먼트                            135
                                     ... 
(주)키다리이엔티,주식회사 키위컴퍼니                    1
소니픽쳐스엔터테인먼트코리아주식회사극장배급지점,(주)키다리이엔티      1
(주)리틀빅픽쳐스,(주)트리니티픽쳐스                    1
(유)조이앤시네마,(주)제이앤씨미디어그룹                  1
TCO(주)더콘텐츠온,(주)전망좋은 영화사                 1
Name: 배급사, Length: 434, dtype: int64

## 2. 데이터 전처리 (배급사 string 분리) 
- 배급사 string을 , 기준으로 row로 분리하여 계산 

In [ ]:
new_movies = movies.assign(배급사=movies.배급사.str.split(","))
new_movies = new_movies.배급사.apply(pd.Series).merge(new_movies, right_index = True, left_index = True)
new_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3375 entries, 0 to 3374
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   0       3374 non-null   object        
 1   1       321 non-null    object        
 2   2       15 non-null     object        
 3   3       1 non-null      object        
 4   4       1 non-null      object        
 5   5       1 non-null      object        
 6   순번      3375 non-null   int64         
 7   개봉연도    3375 non-null   int64         
 8   영화명     3375 non-null   object        
 9   감독      3375 non-null   object        
 10  배급사     3374 non-null   object        
 11  개봉일     3375 non-null   datetime64[ns]
 12  국적      3375 non-null   object        
 13  전국스크린수  3375 non-null   int64         
 14  전국매출액   3375 non-null   int64         
 15  전국관객수   3375 non-null   int64         
 16  장르      3375 non-null   object        
 17  등급      3375 non-null   object        
 18  영화구분    

In [ ]:
new_movies = movies.assign(배급사=movies.배급사.str.split(","))
new_movies = new_movies.배급사.apply(pd.Series).merge(new_movies, right_index = True, left_index = True)
new_new_movies = new_movies.melt(id_vars = ["순번", "개봉연도", "영화명", "감독", "배급사", "개봉일", "국적", "전국스크린수", "전국매출액", "전국관객수", "장르", "등급", "영화구분"], var_name="new_배급사", value_name = "진짜배급사")
new_movies_a = new_new_movies.sort_values(by = ["순번", "new_배급사"], ascending = True)
new_movies = new_movies_a[new_movies_a["진짜배급사"].notnull()]

In [ ]:
new_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3713 entries, 0 to 3374
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   순번       3713 non-null   int64         
 1   개봉연도     3713 non-null   int64         
 2   영화명      3713 non-null   object        
 3   감독       3713 non-null   object        
 4   배급사      3713 non-null   object        
 5   개봉일      3713 non-null   datetime64[ns]
 6   국적       3713 non-null   object        
 7   전국스크린수   3713 non-null   int64         
 8   전국매출액    3713 non-null   int64         
 9   전국관객수    3713 non-null   int64         
 10  장르       3713 non-null   object        
 11  등급       3713 non-null   object        
 12  영화구분     3713 non-null   object        
 13  new_배급사  3713 non-null   object        
 14  진짜배급사    3713 non-null   object        
dtypes: datetime64[ns](1), int64(5), object(9)
memory usage: 464.1+ KB
